In [4]:
import os
import stanza
from tqdm.notebook import tqdm
os.chdir('/home/derek')

To activate the openIE jar, in the directory where OpenIE standalone is located, you have to run:


java -Xmx50g -XX:+UseConcMarkSweepGC -jar openie-assembly-5.0-SNAPSHOT.jar --httpPort 8008

In [10]:
from functools import partial
import pandas as pd
import sys
import warnings
from tqdm import tqdm
import re
import threading
from func_timeout import func_timeout, FunctionTimedOut
import traceback

from pyopenie import OpenIE5
#extractor = OpenIE5('http://localhost:9000')
extractors = [OpenIE5('http://localhost:8000'), OpenIE5('http://localhost:9000')]

import nltk.data
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

import ctypes


total_arash = []
total_rows = []



def HELPER_to_extract(sent, row, core_NLP_client):# = extractors):# None):
    # global total_arash
    # global total_rows
    total_rows = []
    arash = [] # naming one var arash so the legacy lives on...
    try:
        results = core_NLP_client.annotate(sent)
        for res in results:
            for arg2 in res["extraction"]["arg2s"]:
                arash.append([res["extraction"]["arg1"]["text"], \
                    res["extraction"]["rel"]["text"], \
                        arg2["text"], res["confidence"], res["extraction"]["context"], \
                            res["extraction"]["negated"], res["extraction"]["passive"]])
    except:
        return []

    # #with lock:
    #     total_arash.extend(arash)
    for i in range(len(arash)):
        total_rows.append(row)
    return arash, total_rows




def HELPER_clean_sentence(text):
    # TO DO: Refactor with other punctuations...
    text = text.replace('”','"').replace('’','\'').replace(',',',').replace('“','"')
    text1 = re.sub(r'[^\x00-\x7f]',r'', text) 
    return text1.replace("\n"," ")


def HELPER_rels_quick_clean(text):
    # expand appostrophes
    return text#.replace("'s","is").replace("'m","am").replace("'re","are")

# Method to implement extractions(?)
def OPTION1(df = None):
    # global total_arash
    # global total_rows

    df_new = pd.DataFrame()
    headers = ["postnum", "sentencenum", "relnum", "arg1", "rel", "arg2", "confidence", "context", "negated", "passive"]
    partial_sents = []
    partial_rows = []

    for i, row in tqdm(df.iterrows(), total=df.shape[0]):
            
            # This does string preprocessing so it's not gigafucked from trying to adapt pdfs to text.
            sent = str(row["sentence:String"])
            sent1 = sent.replace('”','"').replace('’','\'').replace(',',',').replace('“','"').replace("\n","")
            sent1 = re.sub(r'[^\x00-\x7f]',r'', sent1) 
            if len(sent1) > 300 or "This file" in sent1 or "copyright law" in sent1 or "contributed equally" in sent1 or sent1.count('\n')>1:
                continue

            partial_sents.append(sent1)
            partial_rows.append(row)


            if len(partial_sents) < 1:
                continue
            else:
                # global lock
                # lock = threading.Lock()
                # threads = [threading.Thread(target=HELPER_to_extract, args=(sent, row, extractor)) for (sent,row,extractor) in zip(partial_sents, partial_rows, extractors)]
                # for t in threads: 
                #     t.start()
                # for t in threads:
                #     t.join()
                try:

                    # Here is where the money is made. This is the function that does the relationship extraction. 
                    total_arash, total_rows = HELPER_to_extract(partial_sents[0], partial_rows[0], extractors[0])
                    partial_sents = []
                    partial_rows = []
                except Exception as e:
                    print(e)
                    break#continue

            relationships = total_arash
            total_arash = []

            for k, (rel,r) in enumerate(zip(relationships, total_rows)):
                

                # This step has some ad-hocs to deal with exceptions we know of.
                # TODO: REFACTOR in simpler terms.
                dictionary = {}
                if rel[0] in ["Tcell","T-cell","T-cells","Tcells","t cell","t-cell","T cell", "T cells"]:
                    rel[0] = "T-cell"
                if rel[2] in ["Tcell","T-cell","T-cells","Tcells","t cell","t-cell","T cell", "T cells"]:
                    rel[2] = "T-cell"


                # Constructing the relationships into a format that will eventually be put into a dataframe for neptune upload
                dictionary["arg1:String"] = rel[0]
                dictionary["rel:String"] = HELPER_rels_quick_clean(rel[1])
                dictionary["arg2:String"] = rel[2]
                #dictionary["context"] = sent
                dictionary["confidence:String"] = rel[3]
                dictionary["context_rel:String"] = rel[4]
                dictionary["negated:String"] = rel[5]
                dictionary["passive:String"] = rel[6]

                for k,v in r.items():
                    if k not in ["text","postnum","sentencenum"]:
                        dictionary[k] = v
                try:        
                    df_new = df_new.append(dictionary, ignore_index=True)
                except Exception as e:
                    print(e)
                    continue

            total_rows = []

    #df_new = df_new.reindex(headers, axis=1)
    return df_new



def kill_bill_and_get_extractions(df):
    try:
        df_new = OPTION1(df)
    except Exception as e:
        df_new = pd.DataFrame()
        #tb_str = traceback.format_exc()
        #print(tb_str)
        print(e)
    return df_new


In [8]:
# Imports
import os
import sys
sys.path.append('/home/derek/auto-nlp/auto-nlp-bucket/NLP_dump/')

from PyPDF2 import PdfReader
import PyPDF2
import requests
from bs4 import BeautifulSoup
from scholarly import scholarly
from scholarly import ProxyGenerator
import pandas as pd
import chunk
import boto3
import nltk
from collections import defaultdict
import numpy
from numpy import real
from REL_HELPERS import *
from io import StringIO # python3; python2: BytesIO 
import boto3
import json
import gzip
import multiprocessing
import nltk
from nltk.corpus import stopwords
import random
import string
import spacy
from celery import Celery
from celery.utils.log import get_task_logger
import traceback

ModuleNotFoundError: No module named 'REL_HELPERS'

In [ ]:
# We now iterate over the pubmed articles

total_nodes = pd.DataFrame()
total_edges = pd.DataFrame()
try:
    filepath = os.path.join(input_dir, filename)
    with open(filepath, 'r') as file:
        data_lines = ''.join([line.strip() for line in file.readlines()])

    print(dois.loc[filename[:-4]]['DOI'])
    df = retrieve_metadata(str(data_lines), dois.loc[filename[:-4]]['DOI'])#GET DOI FOR THE PAPER

    # Here is the step for calling the extraction functions
    results = kill_bill_and_get_extractions(df)
    
    # This is to format the results
    temp_node_csv, temp_edge_csv = URANUS_redone(results)
    temp_node_csv["~label"] = "Scientific Literature"
    temp_edge_csv["~label"] = "Scientific Literature"

    total_nodes = total_nodes.append(temp_node_csv, ignore_index=True)
    total_edges = total_edges.append(temp_edge_csv, ignore_index=True)
    print(total_nodes.shape)
    print(total_edges.shape)
    #data_lines.close()
    

    if total_edges.shape[0] > 0: #remove after testing
        # specify the file path and file name
        #file_path_nodes = output_dir+'/df_nodes_{filename}.csv'
        #file_path_edges = output_dir+'/df_edges_{filename}.csv'

        # save the DataFrame to a CSV file
        #total_nodes.to_csv(file_path_nodes, index=False)
        #total_edges.to_csv(file_path_edges, index=False)


        # Save processed data to file in output_dir
        output_filename = f"{filename[:-4]}_processed.csv" # strip ".txt" and add "_processed.csv"
        output_filepath_nodes = os.path.join(output_dir, f"df_nodes_{output_filename}")
        output_filepath_edges = os.path.join(output_dir, f"df_edges_{output_filename}")
        total_nodes.to_csv(output_filepath_nodes, index=False)
        total_edges.to_csv(output_filepath_edges, index=False)
except Exception as e:
    print(f"Error processing {filename}: {e}")
    tb_str = traceback.format_exc()
    print(tb_str)
